In [1]:
import pandas as pd

## My API key  & library_Id are stored in another file
api_key,library_id = pd.read_json("Zotero_API_key.json", typ='series')

library_type="group"




from pyzotero import zotero
zot = zotero.Zotero(library_id, library_type, api_key)

ID_Universities='EWSYI3RS' # to access the files in the university subfolder.

In [2]:
for i in zot.collections_sub(ID_Universities):
    name,key=i["data"]["name"],i["data"]["key"]
    print(f"name: {name} --> key: {key}")

name: OpenUniversiteitNederland --> key: X4HX8ZQE
name: Wageningen University --> key: QMTMEV7G
name: University of Twente --> key: QFGTICY2
name: Utrecht University --> key: 9A2UJIVA
name: Rotterdam University --> key: Z4EJ3VVG
name: University of Groningen --> key: P63XN58M
name: Vrije Universiteit Amsterdam --> key: AZJ9ZEBZ
name: Tilburg University --> key: XF9572P7
name: Eindhoven University of Technoloy --> key: FKNPJ5UD
name: Maastricht Univerisity --> key: QW34VSNN
name: Delft University --> key: FZDYP465
name: University of Amsterdam --> key: 92FECCMX


In [3]:
def relevantItemInfo(dictionary):    
    if dictionary["itemType"]=='attachment':
        # if directory --> key leads to the correct folder
        
        relevant_keys=['key','url']
        newDict={key: dictionary[key] for key in relevant_keys}
        #print(dictionary)
        newDict["note"]=None
       # print(newDict)
        return newDict
    
    if dictionary["itemType"]=="note":
        relevant_keys=['key','note']
        newDict={key: dictionary[key] for key in relevant_keys}
        newDict["url"]=None
        return newDict

    

for i in zot.collection_items(key)[:2]:
    dictionary=i["data"]
    print(dictionary, end="\n\n")
    
    Info=relevantItemInfo(dictionary)
    print(Info, end="\n\n")

{'key': '8KAUPAIY', 'version': 425, 'parentItem': '7ZAUFER2', 'itemType': 'attachment', 'linkMode': 'imported_url', 'title': 'Snapshot', 'accessDate': '2023-05-08T17:09:05Z', 'url': 'https://www.uva.nl/content/evenementen/2023/02/universiteitsforum-ai-in-het-onderwijs.html?cb', 'note': '', 'contentType': 'text/html', 'charset': 'utf-8', 'filename': 'universiteitsforum-ai-in-het-onderwijs.html', 'md5': None, 'mtime': None, 'tags': [], 'relations': {}, 'dateAdded': '2023-05-08T17:09:05Z', 'dateModified': '2023-05-08T17:09:05Z'}

{'key': '8KAUPAIY', 'url': 'https://www.uva.nl/content/evenementen/2023/02/universiteitsforum-ai-in-het-onderwijs.html?cb', 'note': None}

{'key': '7ZAUFER2', 'version': 424, 'itemType': 'webpage', 'title': 'Universiteitsforum: AI in het Onderwijs', 'creators': [{'creatorType': 'author', 'firstName': 'Universiteit van', 'lastName': 'Amsterdam'}], 'abstractNote': 'Het UniversiteitsForum bespreekt maandag 20 februari het onderwerp ‘AI in het Onderwijs’ op Zoom. Ied

In [4]:
dictList=[]

for i in zot.collections_sub(ID_Universities):
    name,key=i["data"]["name"],i["data"]["key"]
    print(f"name: {name} --> key: {key}")
    
    for i in zot.collection_items(key):
        # only use data if there is an attachment.
        if i["data"]["itemType"]=='attachment':
            Info=relevantItemInfo(i["data"])
            dictList+=[{"Uni": name,"FileKey":Info["key"],"url":Info["url"]}]
    


name: OpenUniversiteitNederland --> key: X4HX8ZQE
name: Wageningen University --> key: QMTMEV7G
name: University of Twente --> key: QFGTICY2
name: Utrecht University --> key: 9A2UJIVA
name: Rotterdam University --> key: Z4EJ3VVG
name: University of Groningen --> key: P63XN58M
name: Vrije Universiteit Amsterdam --> key: AZJ9ZEBZ
name: Tilburg University --> key: XF9572P7
name: Eindhoven University of Technoloy --> key: FKNPJ5UD
name: Maastricht Univerisity --> key: QW34VSNN
name: Delft University --> key: FZDYP465
name: University of Amsterdam --> key: 92FECCMX


In [5]:
df=pd.DataFrame(dictList)
df.head()

,Uni,FileKey,url
0,OpenUniversiteitNederland,LURBFR5B,
1,OpenUniversiteitNederland,F32723VK,https://www.ou.nl/en/-/gpt-x-wat-is-het-hoe-we...
2,OpenUniversiteitNederland,RKJ4BNRB,https://www.ou.nl/en/-/symposium-towards-highl...
3,OpenUniversiteitNederland,8G2MCFZJ,https://www.ou.nl/en/-/symposium-towards-highl...
4,OpenUniversiteitNederland,Z5RL79M7,https://www.ou.nl/en/-/onderzoek-onderwijswete...


In [6]:
def FileLinks(Filekey):
   #print(Filekey)
    import os
    DIR=f"C:\\Users\\mr\\Zotero\\storage\\{Filekey}"
    #print(DIR)
    FileNames=os.listdir(DIR)
    file=[i for i in FileNames if i.endswith(".html")]
    return DIR+"\\"+FileNames[-1]



df["file"]=df.FileKey.apply(FileLinks)
df.head()

,Uni,FileKey,url,file
0,OpenUniversiteitNederland,LURBFR5B,,C:\Users\mr\Zotero\storage\LURBFR5B\ChatGPT_in...
1,OpenUniversiteitNederland,F32723VK,https://www.ou.nl/en/-/gpt-x-wat-is-het-hoe-we...,C:\Users\mr\Zotero\storage\F32723VK\gpt-x-wat-...
2,OpenUniversiteitNederland,RKJ4BNRB,https://www.ou.nl/en/-/symposium-towards-highl...,C:\Users\mr\Zotero\storage\RKJ4BNRB\symposium-...
3,OpenUniversiteitNederland,8G2MCFZJ,https://www.ou.nl/en/-/symposium-towards-highl...,C:\Users\mr\Zotero\storage\8G2MCFZJ\symposium-...
4,OpenUniversiteitNederland,Z5RL79M7,https://www.ou.nl/en/-/onderzoek-onderwijswete...,C:\Users\mr\Zotero\storage\Z5RL79M7\onderzoek-...


In [7]:
df.to_json("2023-05-10_Uni_Files_raw.json")

In [8]:
df=pd.read_json("2023-05-10_Uni_Files_raw.json")

# Add Content from HTMLs

In [ ]:
def extract_Image_from_Html(img,target_folder="images",filename="test"):
    filetypes=["png","jpg","jpeg","pdf","svg"]

    
    if img[:4]=="data":
        
        filetype,img=img.split(",",1)
        filetype=filetype.split(";")
        filetype=filetype[0].replace("data:image/","").replace("svg+xml","svg")

        if filetype in filetypes:
            print(filetype)                
            import base64
            img=base64.b64decode(img)
            with open(f"{target_folder}/{filename}.{filetype}", "wb") as fh:
                fh.write(img)
                
            print("image generated.")
        else:
            print(filetype)
        return 
    


def removeIMGs(Content,targetFolder="images",IMG_filename="test"):

    tags=Content.findAll('img')
    
    for count, match in enumerate(tags):
        img=match["src"]
        filename=IMG_filename+"_"+str(count)
        
        extract_Image_from_Html(img,target_folder=targetFolder,filename=filename) 
        
        match['src']=f'deleted: {filename}'
        print(f"image removed: {filename}")
    return 

### to Text the image remove / extract function

FilePath=df.file[100]
with open(FilePath,"r", encoding='utf-8') as f:
            text= f.read()

from bs4 import BeautifulSoup

soup = BeautifulSoup(text, 'html.parser')
Content=soup.find("p").parent
removeIMGs(Content,IMG_filename="UM")

In [10]:
from htmldate import find_date
#find_date("https://www.uu.nl/en/events/chat-gpt-discover-the-magic-of-ai")

def find_dateX(x):
    try:
        return find_date(x)
    except:
        print(x)


df["date"]=df.url.apply(find_dateX)

C:\Users\mr\miniconda3\envs\PyMax\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname CEST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


https://universonline.nl/nieuws/2016/04/28/stage-ook-voor-academici-steeds-belangrijker/
https://universonline.nl/nieuws/2023/03/27/education-after-covid-19-back-to-the-lecture-halls-or-is-online-education-here-to-stay/
https://universonline.nl/nieuws/2023/03/21/onderwijs-na-corona-terug-naar-de-collegebanken-of-is-online-onderwijs-een-blijvertje/
https://publications.aaahq.org/iae/article-pdf/doi/10.2308/ISSUES-2023-013/90291/issues-2023-013.pdf
https://universonline.nl/nieuws/2023/03/21/chatgpt-schrijft-wetenschappelijke-artikelen-en-maakt-huiswerkopdrachten/
https://universonline.nl/nieuws/2022/12/22/the-impact-of-a-new-writing-robot-i-had-chatgpt-take-an-exam-it-got-a-nine/
https://universonline.nl/nieuws/2023/01/20/wat-vinden-tilburgse-studenten-van-chatgpt/
https://tlc.uva.nl/article/chatgpt-workshop-voor-docenten/


In [15]:
import MyLib.nlp as nlp
import MyLib.HTML_prep as HTML_prep
import MyLib.analysis as analysis 
%load_ext autoreload
%autoreload 2

In [30]:

def FileInfo(FilePath):
    Title,h1,h2,text,linkName,linkUrl=None,None,None,None,None,None
    if FilePath.endswith(".html"):
    
        try:
            #print(FilePath)

            with open(FilePath,"r", encoding='utf-8') as f:
                text= f.read()

            from bs4 import BeautifulSoup

            soup = BeautifulSoup(text, 'html.parser')
            
            Title=soup.title.text
            
            #soup=HTML_prep.removeIMGs(soup,IMG_filename=Title.replace(" ","_"))      


            All_divs=[p.parent for p in soup("p") if p.parent.name=="div"]
            All_divs=(list(set(All_divs)))
            All_divs=[HTML_prep.removeIMGs(div,no_img=True) for div in All_divs]        

            h1=[item for sublist in [[i.text for i in div("h1")] for div in All_divs if div("h1")!=None] for item in sublist]
            h2=[item for sublist in [[i.text for i in div("h2")] for div in All_divs if div("h2")!=None] for item in sublist]
            text=[item for sublist in [[i.text for i in div("p")] for div in All_divs if div("p")!=None] for item in sublist]
            linkName=[item for sublist in [[i.text for i in div("a")] for div in All_divs if div("a")!=None] for item in sublist]
            linkUrl=[item for sublist in [[i.get('href') for i in div("a")] for div in All_divs if div("a")!=None] for item in sublist]


        except:
            print(f"error with: {FilePath}",end=". ")
    return pd.Series([Title,h1,h2,text,linkName,linkUrl])
        
        

#df.file[2:6].apply(FileInfo)
df[["Title","h1","h2","text","linkName","linkUrl"]]=df.file.apply(FileInfo)


jpeg
image generated.
image removed: test_0
data:
image removed: test_1
data:
image removed: test_2
jpeg
image generated.
image removed: test_0
data:
image removed: test_1
data:
image removed: test_2
png
image generated.
image removed: test_0
png
image generated.
image removed: test_1
data:
image removed: test_2
data:
image removed: test_3
image removed: test_0
jpeg
image generated.
image removed: test_0
png
image generated.
image removed: test_1
image removed: test_0
image removed: test_1
png
image generated.
image removed: test_0
png
image generated.
image removed: test_0
png
image generated.
image removed: test_0
jpeg
image generated.
image removed: test_0
jpeg
image generated.
image removed: test_0
jpeg
image generated.
image removed: test_0
png
image generated.
image removed: test_0
jpeg
image generated.
image removed: test_0
jpeg
image generated.
image removed: test_0
jpeg
image generated.
image removed: test_0
jpeg
image generated.
image removed: test_0
jpeg
image generated.
ima

In [32]:
df.to_json("2023-05-10_Uni_Files_NLP.json")

In [33]:
def Insert_title(text,Title):
    if Title!=None and type(text)==list:
        text=[Title]+text
        return text
    else:
        return text


df["text"]=df[["text","Title"]].apply(lambda x: Insert_title(*x),axis=1)

In [34]:
ChatGPT_Terms="ChatGPT, GPT3, GPT4, OpenAI, AI, LLM, A.I, GPT, Chatbot, technology, tool, app, Artificial Intelligence, Large Language Models"
ChatGPT_Terms=ChatGPT_Terms.split(", ")
ChatGPT_Terms=[i.lower() for i in ChatGPT_Terms]


In [36]:
def GPTparagraphs(p_list, by=ChatGPT_Terms):
    if p_list!=None:
        AI_Content=[]
        for text in p_list:
            AI_Content_Binary=nlp.WordlistFilter(text, by)
            if AI_Content_Binary==True:
                AI_Content.append(text)
        return AI_Content
            


df["AI_paragraphs"]=df.text.apply(GPTparagraphs)



In [38]:
df2=df[df.file.apply(lambda x: x.endswith("html"))]
df2.text=df2["AI_paragraphs"].apply(lambda x:" ".join(x))

C:\Users\mr\AppData\Local\Temp\ipykernel_17740\2446002319.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.text=df2["AI_paragraphs"].apply(lambda x:" ".join(x))


In [39]:
df2[df2.text.apply(lambda x: type(x))!=str]

,Uni,FileKey,url,file,date,Title,h1,h2,text,linkName,linkUrl,AI_paragraphs


In [40]:
df2["LEN"]=df2.text.apply(lambda x: len(x))
df2=df2[df2.LEN<df2.LEN.max()] ###Cut out the very large list

C:\Users\mr\AppData\Local\Temp\ipykernel_17740\2437524246.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["LEN"]=df2.text.apply(lambda x: len(x))


In [41]:
df2.LEN.describe()

count      134.000000
mean      5703.731343
std      15287.393001
min          0.000000
25%        411.500000
50%       1624.000000
75%       3881.500000
max      96627.000000
Name: LEN, dtype: float64

In [42]:
df3=nlp.NLP_Pipeline(df2, sentiment=False, language="en",translate=True)

cleaning done.
language detection done.
Translating...
nl Aan de han
nl Sinds de r
nl Sinds de r
nl ChatGPT ke
nl Sander Bax
nl ChatGPT sc
nl ChatGPT ka
nl Daarop wer
nl UM zoekend
nl Nieuwe mog
nl Dat maakt 
nl Kijk eens 
nl Universite
nl In haar on
nl Chatbot ho
nl Onze mense
nl ChatGPT in
nl ChatGPT wo
pure english text done. Next: Token & Lemmatizing.
Token & Lemmatizing done. Next: Remove Stopwords.


In [46]:
df3.head()

,Uni,FileKey,url,file,date,Title,h1,h2,text,linkName,...,AI_paragraphs,LEN,text_clean,letters_count,word_count,language,source_language,pure_text,Lemmata,NoStopwords
1,OpenUniversiteitNederland,F32723VK,https://www.ou.nl/en/-/gpt-x-wat-is-het-hoe-we...,C:\Users\mr\Zotero\storage\F32723VK\gpt-x-wat-...,2022-05-12,"GPT-x: Wat is het, hoe werkt het en wat beteke...",[],"[Aanmeldinformatie, Praktische gegevens, Over ...",Aan de hand van twee inleidende presentaties v...,"[aanmeldformulier, Cursus- en vergadercentrum ...",...,[Aan de hand van twee inleidende presentaties ...,792,Based on two introductory presentations by Pro...,788,124,en,nl,based on two introductory presentations by pro...,"[base, on, two, introductory, presentation, by...","[base, introductory, presentation, prof, anna,..."
2,OpenUniversiteitNederland,RKJ4BNRB,https://www.ou.nl/en/-/symposium-towards-highl...,C:\Users\mr\Zotero\storage\RKJ4BNRB\symposium-...,2022-05-12,Symposium: Towards Highly-Informative Learning...,[],"[AI in het onderwijs, AI-kunst, Het programma,...",Sinds de release van ChatGPT in november 2022 ...,"[zijn AI-kunst, www.ou.nl/uitnodiging, cookies...",...,[Sinds de release van ChatGPT in november 2022...,2217,"Since the release of ChatGPT in November , Art...",2195,324,en,nl,since the release of chatgpt in november arti...,"[since, the, release, of, chatgpt, in, novembe...","[release, chatgpt, november, artificial, intel..."
3,OpenUniversiteitNederland,8G2MCFZJ,https://www.ou.nl/en/-/symposium-towards-highl...,C:\Users\mr\Zotero\storage\8G2MCFZJ\symposium-...,2022-05-12,Symposium: Towards Highly-Informative Learning...,[],"[AI in het onderwijs, AI-kunst, Het programma,...",Sinds de release van ChatGPT in november 2022 ...,"[zijn AI-kunst, www.ou.nl/uitnodiging]",...,[Sinds de release van ChatGPT in november 2022...,2217,"Since the release of ChatGPT in November , Art...",2195,324,en,nl,since the release of chatgpt in november arti...,"[since, the, release, of, chatgpt, in, novembe...","[release, chatgpt, november, artificial, intel..."
4,OpenUniversiteitNederland,Z5RL79M7,https://www.ou.nl/en/-/onderzoek-onderwijswete...,C:\Users\mr\Zotero\storage\Z5RL79M7\onderzoek-...,2018-10-26,Projecten - Open Universiteit - Open Universiteit,[],[],,"[cookies, privacyverklaring]",...,[],0,,0,0,en,no_language_features,,[],[]
5,Wageningen University,UC9RHT5Z,https://www.wur.nl/en/newsarticle/can-you-cred...,C:\Users\mr\Zotero\storage\UC9RHT5Z\can-you-cr...,2023-02-06,Can you credit ChatGPT as a co-author? - WUR,[Can you credit ChatGPT as a co-author?],"[Value Creation & Cooperation, What is ChatGPT...",Can you credit ChatGPT as a co-author? - WUR M...,"[Value Creation & Cooperation, Collaborating, ...",...,"[Can you credit ChatGPT as a co-author? - WUR,...",9021,Can you credit ChatGPT as a co-author? - WUR M...,8964,1437,en,en,can you credit chatgpt as a coauthor wur many...,"[can, you, credit, chatgpt, as, a, coauthor, w...","[credit, chatgpt, coauthor, wur, scientist, pl..."


In [43]:
df3.to_json("2023-05-10_Zotero_AI_nlp_en.json")

In [44]:
Lists=df3.NoStopwords.to_list()
Lists=[i for i in Lists if i!=None]
Words=[item for sublist in Lists for item in sublist]

from collections import Counter
a_counter = Counter(Words)
most_common = a_counter.most_common(10)

most_common

[('ai', 1502),
 ('chatgpt', 1430),
 ('text', 1068),
 ('use', 857),
 ('student', 754),
 ('writer', 417),
 ('information', 404),
 ('education', 384),
 ('source', 351),
 ('generate', 351)]

In [45]:
from ipywidgets import interactive, interact

@interact(column=["NoStopwords"],Mostcommon=(5,200))
def make_wordcloud(column="NoStopwords",Mostcommon=20,Save_as=""):
    hashtags=[i.lower() for s in df3[column].dropna() for i in s] # hashtags OR # no stopwords
    analysis.make_wordcloud(hashtags,filename=Save_as,Mostcommon=Mostcommon)

interactive(children=(Dropdown(description='column', options=('NoStopwords',), value='NoStopwords'), IntSlider…

In [ ]:
df3.to_excel("2023-05-08_ZoteroHTMLs.xlsx")